In [109]:
import numpy as np

In [150]:
#对于dat和txt文件的处理是一样的，dat文件后缀改成txt就行，就可以用下面的处理方式
f=np.loadtxt("hw2_lssvm_all.txt")
print(f.shape)
X=f[:,:10]                              #本来就是考虑z域，不需要管x0的问题
X_train=X[0:400,:]                        #X_train dim为400*10
X_test=X[400:,:]                          #X_test dim为600*10

y=f[:,10]
y_train=y[0:400]
y_test=y[400:]

(500, 11)


In [182]:
class KRR(object):
    def __init__(self,lam,gamma):
        self.lam=lam
        self.gamma=gamma
        
    def gaussian_kernel(self,X):
        N=X.shape[0]
        kernel=np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                norm=np.linalg.norm(X[i,:]-X[j,:],2)
                kernel[i][j]=np.exp(-gamma*norm)           #kernel dim为 N*N
        return kernel
        
    def getBelta(self,X,y):         #这就是llsvm
        eye=np.eye(y.shape[0])
        return np.linalg.pinv(self.lam*eye+self.gaussian_kernel(X)).dot(y)                    #belta的dim为N*1
    
    def Ein(self,X_train,y_train,belta):            #由于belta的dim是400*1的，对于Y_train（dim为400*1）的预测只需要正常矩阵运算就行
        kernel=self.gaussian_kernel(X_train)
        predict_y=np.sign(np.dot(kernel,belta))           #怎么写y_predict很重要！！！
        return sum(predict_y!=y_train)/y_train.shape[0]
     
    def Eout(self,X_train,X_test,y_test,belta):         #在预测Y_test时（dim为100*1），由于belta的dim是400*1的，不能直接矩阵运算
        y_predict=[]
        for i in range(X_test.shape[0]):       #对X_test的每个数据分别处理
            x=X_test[i,:]          #x的dim为1*10
            y=0
            for j in range(belta.shape[0]):
                y+=belta[j]*np.exp(-self.gamma*sum((x-X_train[j])**2))            # X_train中的每个数据都要跟x进行操作
            y=sign(y)
            y_predict.append(y)
        return sum(y_predict!=y_test)/y_test.shape[0]

In [180]:
if __name__ == '__main__':
    #第19题
    Ein_min=10
    lam_best=0
    gamma_best=0
    for lam in (0.001,1,1000):
        for gamma in (32,2,0.125):
            model=KRR(lam,gamma)
            belta=model.getBelta(X_train,y_train)

            Ein=model.Ein(X_train,y_train,belta)
            print('Ein',Ein,'lambda=',lam,'gamma=',gamma)
            if Ein<Ein_min:
                Ein_min=Ein
                lam_best=lam
                gamma_best=gamma
    print('Ein_min=',Ein_min,'lambda=',lam_best,'gamma=',gamma_best)
#发现，大部分Ein都是0，说明这个算法的拟合非常到位。

Ein 0.0 lambda= 0.001 gamma= 32
Ein 0.0 lambda= 0.001 gamma= 2
Ein 0.0 lambda= 0.001 gamma= 0.125
Ein 0.0 lambda= 1 gamma= 32
Ein 0.0 lambda= 1 gamma= 2
Ein 0.1525 lambda= 1 gamma= 0.125
Ein 0.0 lambda= 1000 gamma= 32
Ein 0.0 lambda= 1000 gamma= 2
Ein 0.3175 lambda= 1000 gamma= 0.125
Ein_min= 0.0 lambda= 0.001 gamma= 32


In [183]:
if __name__ == '__main__':
    #第20题
    Eout_min=10
    lam_best=0
    gamma_best=0
    for lam in (0.001,1,1000):
        for gamma in (32,2,0.125):
            model=KRR(lam,gamma)
            belta=model.getBelta(X_train,y_train)
            
            Eout=model.Eout(X_train,X_test,y_test,belta)
            print('Eout',Eout,'lambda=',lam,'gamma=',gamma)
            if Eout<Eout_min:
                Eout_min=Eout
                lam_best=lam
                gamma_best=gamma
    print('Eout_min=',Eout_min,'lambda=',lam_best,'gamma=',gamma_best)

Eout 0.44 lambda= 0.001 gamma= 32
Eout 0.44 lambda= 0.001 gamma= 2
Eout 0.58 lambda= 0.001 gamma= 0.125
Eout 0.44 lambda= 1 gamma= 32
Eout 0.44 lambda= 1 gamma= 2
Eout 0.53 lambda= 1 gamma= 0.125
Eout 0.44 lambda= 1000 gamma= 32
Eout 0.44 lambda= 1000 gamma= 2
Eout 0.39 lambda= 1000 gamma= 0.125
Eout_min= 0.39 lambda= 1000 gamma= 0.125
